In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pymysql

def get_html(url):
    web=webdriver.Chrome()
    web.get(url)
    time.sleep(1)
    web.find_element(by=By.XPATH,value='//div[@id="hotBook"]/div[2]/div[1]/span[9]').click()
    time.sleep(1)
    return web.find_elements(by=By.XPATH,value='//div[@class="list"]/div')
    
info=[]
def parser(obj):
    for row in obj:
        b_name=row.find_element(by=By.TAG_NAME,value='p').text#书名
        i_url=row.find_element(by=By.TAG_NAME,value='img').get_attribute('src')#图片链接
        b_id=1#图书ID
        info.append((b_name,i_url,b_id))

def save_mysql(sql,val,**dbinfo):  # 将爬取的内容存入mysql数据库
    try:
        con = pymysql.connect(**dbinfo)  # 连接mysql数据库
        cur = con.cursor()  # 创建游标对象
        cur.executemany(sql,val)
        con.commit()
    except Exception as e:  # 若报错，则获得错误类型
        con.rollback()
        print(e)
    finally:
        cur.close()  # 关闭游标
        con.close()  # 关闭链接
    
if __name__=='__main__':
    url='https://www.ptpress.com.cn/shopping/index'
    obj=get_html(url)
    parser(obj)

    parms={
        "host":"localhost",
        "user":"root",
        "password": "root",
        "database":"study",
        "port":3306
    }
    sql='insert into youdian(b_name,i_url,b_id) values(%s,%s,%s)'
    save_mysql(sql,info,**parms)
    print("已存储全部数据！")


已存储全部数据！
